In [1]:
import os
import time
from tkinter import messagebox, Tk
import webbrowser

import cv2
import numpy as np

In [2]:
def print_date_time():
    """Updates current date and time and keys info on to video."""
    current_time = time.asctime()
    cv2.putText(frame, str(current_time), (280, 24),
                font, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, '   Press h for options : Threshold = '
                + str(thresh) + ' : Save detected images is: '
                + str(md_switch),
                (10, 470), font, 0.5, (255, 255, 255), 1)
    
    


In [3]:
def open_folder():
    """open systems file browser to view images folder."""
    cwd = os.getcwd()
    webbrowser.open(cwd)


In [4]:
root = Tk()
motion_detect = 0
md_switch = 'OFF'
my_dir = r'D:\IDS 1.0\DET_IMGS'
check_folder = os.path.isdir(my_dir)
# If folder doesn't exist, then create it.
if not check_folder:
    os.makedirs(my_dir)
# Make that folder current dir.
os.chdir(my_dir)
thresh=1600
# Used to count individualy named frames as jpgs.
img_index = 0
font = cv2.FONT_HERSHEY_SIMPLEX


# Set up the video capture using the default webcam
video_capture = cv2.VideoCapture(0)

# Initialize the background subtractor and the base frame
background_subtractor = cv2.createBackgroundSubtractorMOG2()
base_frame = None
base_frame_interval = 10  # Interval in seconds (2 minutes)

# Timer variables
timer_start = cv2.getTickCount()

while True:
    # Read the current frame from the video capture
    ret, frame = video_capture.read()
    keyPress = cv2.waitKey(20)
    # Check if it's time to update the base frame
    timer_elapsed = (cv2.getTickCount() - timer_start) / cv2.getTickFrequency()
    if base_frame is None or timer_elapsed >= base_frame_interval:
        # Update the base frame
        base_frame = frame.copy()
        timer_start = cv2.getTickCount()  # Reset the timer
        print("Base frame has been updated!")

    # Apply background subtraction to obtain the foreground mask
    fg_mask = background_subtractor.apply(frame)

    # Perform any necessary preprocessing on the foreground mask (e.g., filtering, thresholding)

    # Find contours of the detected objects
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Calculate the area of the contour
        contour_area = cv2.contourArea(contour)

        # Set a threshold to filter out small contours
        if contour_area > thresh:
            # Draw a bounding box around the detected object
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, 'MD ' + str(img_index),
                    (0, 20), font, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            print_date_time()
            if motion_detect == 1:
                frame_name = (str(img_index) + str('.jpg'))
                cv2.imwrite(frame_name, frame)
                img_index += 1

    print_date_time()
    cv2.imshow('Live video', frame)
    
    # Enter key pauses video stream.
    if keyPress & 0xFF == 13:
        cv2.putText(frame, 'OPERATION PAUSED', (210, 260), font, 2,
                    (0, 255, 0), 8, cv2.LINE_AA)
        cv2.imshow('Live video', frame)
        cv2.waitKey(0)

    # q key to quit program.
    if keyPress & 0xFF == ord('q'):
        root.withdraw()
        ask_yn = messagebox.askyesno('Quit Motion Detector?',
                                     'Are you sure?')
        if ask_yn:
            break
        root.update_idletasks()

    # Motion detect off. s key.
    if keyPress & 0xFF == ord('s'):
        motion_detect = 0
        md_switch = 'OFF'
        img_index=0

    # Motion detect on. m key.
    if keyPress & 0xFF == ord('m'):
        motion_detect = 1
        md_switch = 'ON'

    # Camera sensitivity + key.
    if keyPress & 0xFF == ord('+'):
        thresh += 50

    # Camera sensitivity - key.
    if keyPress & 0xFF == ord('-'):
        thresh -= 50

    # View images folder v key.
    if keyPress & 0xFF == ord('v'):
        open_folder()

    # Snapshot x key.
    if keyPress & 0xFF == ord('x'):
        frame_name = (str(img_index) + str('-snapshot.jpg'))
        cv2.imwrite(frame_name, frame)
        img_index += 1

    # Help keys msg box.
    if keyPress & 0xFF == ord('h'):
        root.withdraw()
        messagebox.showinfo('Intrusion Detection System help - Keys',
                            'H ~ This menu\n\n'
                            'M ~ Start motion dectect\n\n'
                            'S ~ Stop motion detect\n\n'
                            'X ~ Take a single snaphot\n\n'
                            'V ~ View images folder\n\n'
                            '+ ~ Camera sensitivity increase\n\n'
                            '- ~ Camera sensitivity deccrease\n\n'
                            'Q ~ Quit\n\n'
                            'ENTER - Pause video stream\n\n'
                            'Motion Detector V2 is Freeware.\n'
                            'By KK And Jose, June 2023.\n\n'
                            
                            )

# Close down.
video_capture.release()
cv2.destroyAllWindows()

root.mainloop()


Base frame has been updated!
Base frame has been updated!
Base frame has been updated!
Base frame has been updated!
Base frame has been updated!
Base frame has been updated!
Base frame has been updated!
Base frame has been updated!
Base frame has been updated!
